In [1]:
# Import dependencies
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()
from datetime import datetime, date

In [2]:
# Import configuration variables
# import sys
# sys.path.append('../../../../')
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [3]:
# Set up simple_salesforce
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [4]:
# Connect to your MySQL database
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [5]:
# Query Data
class_data = pd.read_sql("SELECT * FROM class", conn)
class_data.head()

,ID_Class,ID_Course,Section,StartDate,EndDate
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


## Query Course data from Salesforce in order to Get its 'Record Id'

In [6]:
sf_class_data = sf.query_all_iter("SELECT Id, ID_Course__c FROM Course__c")

In [7]:
class_sf = []

for data in sf_class_data:
    records = {
        'Course_Id_record': data['Id'],
        'ID_Course__c': data['ID_Course__c']
    }
    
    class_sf.append(records)

In [8]:
class_sf

[{'Course_Id_record': 'a033h000006VstvAAC', 'ID_Course__c': '1'},
 {'Course_Id_record': 'a033h000006VstwAAC', 'ID_Course__c': '2'},
 {'Course_Id_record': 'a033h000006VstxAAC', 'ID_Course__c': '3'},
 {'Course_Id_record': 'a033h000006VstyAAC', 'ID_Course__c': '4'},
 {'Course_Id_record': 'a033h000006Vsu0AAC', 'ID_Course__c': '6'},
 {'Course_Id_record': 'a033h000006Vsu1AAC', 'ID_Course__c': '7'},
 {'Course_Id_record': 'a033h000006Vsu2AAC', 'ID_Course__c': '8'},
 {'Course_Id_record': 'a033h000006VstzAAC', 'ID_Course__c': '5'},
 {'Course_Id_record': 'a033h000006VgchAAC', 'ID_Course__c': '3'},
 {'Course_Id_record': 'a033h000006VgcmAAC', 'ID_Course__c': '4'},
 {'Course_Id_record': 'a033h000006Vgd1AAC', 'ID_Course__c': '8'},
 {'Course_Id_record': 'a033h000006ViArAAK', 'ID_Course__c': '1'},
 {'Course_Id_record': 'a033h000006ViAsAAK', 'ID_Course__c': '2'},
 {'Course_Id_record': 'a033h000006ViAtAAK', 'ID_Course__c': '3'},
 {'Course_Id_record': 'a033h000006ViAuAAK', 'ID_Course__c': '4'},
 {'Course_

In [9]:
class_sf = pd.DataFrame(class_sf)
class_sf.head()

,Course_Id_record,ID_Course__c
0,a033h000006VstvAAC,1
1,a033h000006VstwAAC,2
2,a033h000006VstxAAC,3
3,a033h000006VstyAAC,4
4,a033h000006Vsu0AAC,6


In [10]:
class_sf['ID_Course__c'] = class_sf['ID_Course__c'].astype(int)

In [11]:
class_data['StartDate'] = class_data['StartDate'].apply(lambda x: x.strftime('%Y-%m-%d'))
class_data['EndDate'] = class_data['EndDate'].apply(lambda x: x.strftime('%Y-%m-%d')) 
print(class_data)

   ID_Class  ID_Course               Section   StartDate     EndDate
0         1          1  GWU-ARL-DATA-PT-09-0  2020-03-16  2020-03-14
1         2          1       GWDC201805DATA3  2018-05-15  2018-11-08
2         3          2       GWARL201905WEB3  2019-05-14  2019-11-07
3         4          3      GWARL201905UIUX3  2019-05-14  2019-11-07


In [12]:
# Transform Data
class_data = class_data.rename(columns={
                                        'ID_Class':'ID_Class__c',
                                        'ID_Course':'ID_Course__c',
                                        'Section':'Section__c',
                                        'StartDate':'StartDate__c',
                                        'EndDate':'EndDate__c'
                                })
class_data.head()

,ID_Class__c,ID_Course__c,Section__c,StartDate__c,EndDate__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [13]:
class_data_sf = class_data.merge(class_sf)
class_data_sf

,ID_Class__c,ID_Course__c,Section__c,StartDate__c,EndDate__c,Course_Id_record
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14,a033h000006VstvAAC
1,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14,a033h000006ViArAAK
2,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14,a033h000006VgcXAAS
3,2,1,GWDC201805DATA3,2018-05-15,2018-11-08,a033h000006VstvAAC
4,2,1,GWDC201805DATA3,2018-05-15,2018-11-08,a033h000006ViArAAK
5,2,1,GWDC201805DATA3,2018-05-15,2018-11-08,a033h000006VgcXAAS
6,3,2,GWARL201905WEB3,2019-05-14,2019-11-07,a033h000006VstwAAC
7,3,2,GWARL201905WEB3,2019-05-14,2019-11-07,a033h000006ViAsAAK
8,3,2,GWARL201905WEB3,2019-05-14,2019-11-07,a033h000006VgccAAC
9,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07,a033h000006VstxAAC


In [14]:
# Convert DataFrame to a list of dictionaries --> This can help facilitate a bulk insert
class_load = class_data_sf.to_dict(orient='record')
class_load

[{'ID_Class__c': 1,
  'ID_Course__c': 1,
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'StartDate__c': '2020-03-16',
  'EndDate__c': '2020-03-14',
  'Course_Id_record': 'a033h000006VstvAAC'},
 {'ID_Class__c': 1,
  'ID_Course__c': 1,
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'StartDate__c': '2020-03-16',
  'EndDate__c': '2020-03-14',
  'Course_Id_record': 'a033h000006ViArAAK'},
 {'ID_Class__c': 1,
  'ID_Course__c': 1,
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'StartDate__c': '2020-03-16',
  'EndDate__c': '2020-03-14',
  'Course_Id_record': 'a033h000006VgcXAAS'},
 {'ID_Class__c': 2,
  'ID_Course__c': 1,
  'Section__c': 'GWDC201805DATA3',
  'StartDate__c': '2018-05-15',
  'EndDate__c': '2018-11-08',
  'Course_Id_record': 'a033h000006VstvAAC'},
 {'ID_Class__c': 2,
  'ID_Course__c': 1,
  'Section__c': 'GWDC201805DATA3',
  'StartDate__c': '2018-05-15',
  'EndDate__c': '2018-11-08',
  'Course_Id_record': 'a033h000006ViArAAK'},
 {'ID_Class__c': 2,
  'ID_Course__c': 1,
  'Section__c': 'GWDC201805DA

In [15]:
'''
# Inserting rows in bulk for Class table on Salesforce
try:
    sf.bulk.class__c.insert(tryout1)
except Exception as e:
    print(e)
'''

'\n# Inserting rows in bulk for Class table on Salesforce\ntry:\n    sf.bulk.class__c.insert(tryout1)\nexcept Exception as e:\n    print(e)\n'

In [16]:
for row in class_load:
    
    record = {
        'ID_Class__c': row['ID_Class__c'],
        'ID_Course__c': row['ID_Course__c'],
        'Section__c':row['Section__c'],
        'StartDate__c':row['StartDate__c'],
        'EndDate__c':row['EndDate__c']
    }
    try:
        sf.class__c.create(record)
    except Exception as e:
        print(e)